In [ ]:
import os, sys
from scipy import stats
import numpy as np
import pandas as pd

from scipy import stats

is_colab = False

if is_colab:
    # create src and upload stat_lib into it
    from src import stat_lib
else:
    sys.path.insert(1, '../src/')
    import stat_lib

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# em stat_lib --> definição do que é colors
stat_lib.colors

### Medindo a glicemina no lab e apresentando resultados

O que você faz no lab?

> Primeiro, você convoca 30 pacientes (escolher - critérios inclusão x exclusão)   
> Explica o experimento que está fazendo e pede autorização (consentimento assistido).  
> Em um determinado dia você recolhe amostras para exame de glicemia (antes do café da manhã, em jejum)  
> Depois, medica os pacientes.  
> Passados 24 h você recolhe novas amostras, em jejum, e faz a comparação.

**Qual o erro neste desenho experimental???**

> Faltou o placebo ! O falso medicamento  
> Effeito "psicológico"  
> Tem que demonstrar que os Pacientes que tomaram Placebo estão próximos do Controle (antes do tratamento)  
> Os pacientes medicados devem ter um Tamanho de Efeito sinsível
> Os novos paciente que tomaram Placebo (sem saber: cego = *blind*) não devem apresentar Tamanho de Efeito
> **Criticar**: podemos fazer um experimento com placebo?  
> Ainda poderia ter um Controle Positivo: uma droga anti-glicêmica de mercado!  


### Ver a Declaração de Helsinki:

https://en.wikipedia.org/wiki/Declaration_of_Helsinki

https://www.wma.net/policies-post/wma-declaration-of-helsinki/

In [ ]:
MUc = 140; SSDc = 15
MUh = 110; SSDh = 15
MUm = 105; SSDm = 12
N   = 30

sampC = np.random.normal(loc=MUc, scale=SSDc, size=N)
sampH = np.random.normal(loc=MUh, scale=SSDh, size=N)
sampM = np.random.normal(loc=MUm, scale=SSDm, size=N)

mu_c = np.mean(sampC); ssd_c = np.std(sampC)
mu_h = np.mean(sampH); ssd_h = np.std(sampH)
mu_m = np.mean(sampM); ssd_m = np.std(sampM)

### O que vocẽ faz no lab? No Prisma ...

In [ ]:
df = pd.DataFrame({'control': sampC, 'masc': sampH, 'fem': sampM})
df.head()

## Empilhamento (stack) - valor x classe

In [ ]:
dfc = pd.DataFrame({'val': sampC, 'grupo': 'control'})
dfc.head(3)

In [ ]:
dfc = pd.DataFrame({'val': sampC, 'grupo': 'control'})
dfh = pd.DataFrame({'val': sampH, 'grupo': 'masc'})
dfm = pd.DataFrame({'val': sampM, 'grupo': 'fem'})

dfs = pd.concat([dfc, dfh, dfm])
dfs.reset_index(inplace=True, drop=True)

dfs.shape, dfs['grupo'].unique()                   

In [ ]:
dfs.head(3)

In [ ]:
dfs.tail(3)

### Barplot com Seaborn

https://seaborn.pydata.org/generated/seaborn.barplot.html

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

colors = ['yellow', 'blue', 'red']

ax = sns.barplot(x="grupo", hue="grupo", y="val", data=dfs, saturation=0.6, palette=colors, errorbar=('ci', 95), ax=ax)
plt.title("Tratamento de Glicemia")
plt.ylabel('glicemia (mg/dl)')
plt.xlabel('')

In [ ]:
t_H, pval_H, text_H = stat_lib.calc_ttest_independente(sampC, sampH, equal_var=False)
print(text_H)

In [ ]:
t_M, pval_M, text_M = stat_lib.calc_ttest_independente(sampC, sampM, equal_var=False)
print(text_M)

In [ ]:
t_e, pval_e, text_e = stat_lib.calc_ttest_independente(sampH, sampM, equal_var=False)
print(text_e)

### Significância

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['yellow', 'blue', 'red']

ax = sns.barplot(x="grupo", hue="grupo", y="val", data=dfs, saturation=0.6, palette=colors, errorbar=('ci', 95), ax=ax)

ax.set_title("Tratamento de Glicemia com n = %d"%(N))
ax.set_ylabel('glicemia (mg/dl)')
ax.set_xlabel('grupos')

y1 = 150; del_y1 = 5; xmin=0; xmax=1
plt.hlines(y=y1, xmin=xmin, xmax=xmax, colors='black')
plt.text(x=.2, y=y1+del_y1, s=text_H)

y2 = 170; del_y2 = 5; xmin=0; xmax=2
plt.hlines(y=y2, xmin=xmin, xmax=xmax, colors='black')
plt.text(x=.2, y=y2+del_y2, s=text_M)

plt.ylim(0, 200);

### As distribuições são nomais? teste de Shapiro-Wilkis

In [ ]:
retNormC, textNormC, text_NormC, statNormC, pvalueNormC = stat_lib.calc_normalidade_SWT(sampC, 0.05)
textNormC, text_NormC

In [ ]:
retNormH, textNormH, text_NormH, statNormH, pvalueNormH = stat_lib.calc_normalidade_SWT(sampH, 0.05)
textNormH, text_NormH

In [ ]:
retNormM, textNormM, text_NormM, statNormM, pvalueNormM = stat_lib.calc_normalidade_SWT(sampM, 0.05)
textNormM, text_NormM

### ANOVA - análise de variância (Ronald Fisher)

In [ ]:
stats.f_oneway(sampC, sampH, sampM)

In [ ]:
# está no stat_lib

lista_de_amostras = [sampC, sampH, sampM]
ret, text, text_, stat, pvalue = stat_lib.test_one_way_ANOVA_list(lista_de_amostras, alpha = 0.05)
text, text_

### Mas quem é diferente??? Dunnett responde.

  - Teste de Dunnet:
    - vários casos
    - uma referência

In [ ]:
res, pvals = stat_lib.calc_Dunnett_test([sampH, sampM],  control=sampC)
pvals

### Repare que gerou 2 p-valores
  - pvals[0] - sampH (homens) x sampC (controle)
  - pvals[1] - sampM (mulheres) x sampC (controle)


### Construindo os gráficos passo a passo

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

color='green'
ax = sns.histplot(sampC, stat='density', color=color, alpha=.2, label='controle', ax=ax)
sns.rugplot(sampC, color=color, alpha=0.4, ax=ax)

color='blue'
ax = sns.histplot(sampH, stat='density', color=color, alpha=.2, label='homem', ax=ax)
sns.rugplot(sampH, color=color, alpha=0.4, ax=ax)

color='red'
ax = sns.histplot(sampM, stat='density', color=color, alpha=.2, label='mulher', ax=ax)
sns.rugplot(sampM, color=color, alpha=0.4, ax=ax)

plt.grid()
plt.legend();

In [ ]:
colors = ['green', 'blue', 'red']
labels = ['controle', 'homem', 'mulher']

samp_list = [sampC, sampH, sampM]

seqx = np.linspace(80, 180, 100)

fig, ax = plt.subplots(figsize=(10,8))

for i in range(3):
    # ax = sns.histplot(sampC, stat='density', color=color, alpha=.2, label='controle', ax=ax)
    sample = samp_list[i]
    color = colors[i]
    label = labels[i]

    pdf_normal = stats.norm.pdf(seqx, np.mean(sample), np.std(sample))
    sns.lineplot(x=seqx, y=pdf_normal, color=color, label=label, ax=ax)

ax.set_xlabel('glicemia mg/dl')
ax.set_ylabel('probabiidade')

title = f"Tratamento de Glicemia com {N} amostras nos 3 grupos"
ax.set_title(title)
plt.grid()
plt.legend();

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(14,12), sharex=True)

ax = axes[0]
ax = sns.barplot(y='grupo', x="val", data=dfs, hue='grupo', saturation=0.6, palette=colors, errorbar=('ci', 95), ax=ax)

ax.set_ylabel('grupos')

title = f"Tratamento de Glicemia com {N} amostras nos 3 grupos"
ax.set_title(title)

ax = axes[1]
for i in range(3):
    # ax = sns.histplot(sampC, stat='density', color=color, alpha=.2, label='controle', ax=ax)
    sample = samp_list[i]
    color = colors[i]
    label = labels[i]

    pdf_normal = stats.norm.pdf(seqx, np.mean(sample), np.std(sample))
    sns.lineplot(x=seqx, y=pdf_normal, color=color, label=label, ax=ax)

#ax.set_ylabel('glicemia mg/dl')
ax.set_xlabel('probabiidade')

title = 'Três distribuições'
ax.set_title(title)

pval = pvals[0]
aster = stat_lib.stat_asteristics(pval)
xmin=MUh
xmax=MUc
y=0.032
x=xmin

ax.hlines(y=y, xmin=xmin, xmax=xmax, colors='navy')
ax.annotate(f'{aster} pvalue={pval:.2e}', xy=(x,y), xytext=(x+3, y+0.001), color = 'black')

pval = pvals[1]
aster = stat_lib.stat_asteristics(pval)
xmin=MUm
xmax=MUc
y=0.036
x=xmin

ax.hlines(y=0.036, xmin=xmin, xmax=xmax, colors='darkred')
ax.annotate(f'{aster} pvalue={pval:.2e}', xy=(x,y), xytext=(x+3, y+0.001), color = 'black')
ax.set_ylim(0, 0.05)
ax.set_ylabel('probabilidade')

ax = axes[2]
#   showmeans=False, showmedians=True, showextrema=True
vio_colors = ['lightgreen', 'lightblue', 'pink']
sns.violinplot(y="grupo", x="val", hue='grupo', data=dfs, saturation=0.6, palette=vio_colors, inner=None, ax=ax)
# jitter - pontos internos
sns.stripplot(x="val", y="grupo", data=dfs, color="black", jitter=True, alpha=0.4, size=4, ax=ax)

ax.set_ylabel('grupos')
ax.set_xlabel('glicemia (mg/dl)')
ax.set_title("violin-plot");
